# Quantitative Value Investing
--> build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

--> filter for stocks with the lowest percentiles on the following metrics:

Price-to-earnings ratio
Price-to-book ratio
Price-to-sales ratio
Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
Enterprise Value divided by Gross Profit (EV/GP)

## 1) Import all libraries

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

Import list of stocks from csv file and import API token

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

Determine portfolio size (user input)

In [3]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        
portfolio_input()

Enter the value of your portfolio:250000


## 2) Test for 1 symbol

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data
#data is returned as dictionary of key:value

{'avgTotalVolume': 78693954,
 'calculationPrice': 'close',
 'change': 1.77,
 'changePercent': 0.01217,
 'close': 152.4,
 'closeSource': 'faoiicfl',
 'closeTime': 1642667858140,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 149.29,
 'delayedPriceTime': 1701100067874,
 'extendedChange': 0.24,
 'extendedChangePercent': 0.0017,
 'extendedPrice': 148.03,
 'extendedPriceTime': 1695407849655,
 'high': 152.4,
 'highSource': 'die5enpaermudi 1te c ly',
 'highTime': 1656509014539,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 152.59,
 'iexBidSize': 129,
 'iexClose': 149.43,
 'iexCloseTime': 1673263063708,
 'iexLastUpdated': 1705542504629,
 'iexMarketPercent': 0.011775180557636659,
 'iexOpen': 151.58,
 'iexOpenTime': 1638732913226,
 'iexRealtimePrice': 150.74,
 'iexRealtimeSize': 26,
 'iexVolume': 1006987,
 'lastTradeTime': 1659729024477,
 'latestPrice': 151.3,
 'latestSource': 'Close',
 'latestTime': 'October 18, 2021',
 'latestUpdate': 1665052152321,
 'latestVolume': 8

## 3) Execute a batch API and build DataFrame
--> to make the call run faster and more efficiently, we make batches of 100 symbols max.
--> to do that run a function called chunks which will create groups of 100 symbols.
--> create a blank DataFrame and add our data to the data frame one-by-one.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100)) # "Ticker" is from the csv file
print(symbol_groups)


[0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
      ...  
95     CINF
96       CL
97      CLX
98      CMA
99    CMCSA
Name: Ticker, Length: 100, dtype: object, 100     CME
101     CMG
102     CMI
103     CMS
104     CNC
       ... 
195    FTNT
196     FTV
197      GD
198      GE
199    GILD
Name: Ticker, Length: 100, dtype: object, 200     GIS
201      GL
202     GLW
203      GM
204    GOOG
       ... 
295     MAA
296     MAR
297     MAS
298     MCD
299    MCHP
Name: Ticker, Length: 100, dtype: object, 300     MCK
301     MCO
302    MDLZ
303     MDT
304     MET
       ... 
395     RHI
396     RJF
397      RL
398     RMD
399     ROK
Name: Ticker, Length: 100, dtype: object, 400     ROL
401     ROP
402    ROST
403     RSG
404     RTX
       ... 
495    XLNX
496     XOM
497    XRAY
498     XRX
499     XYL
Name: Ticker, Length: 100, dtype: object, 500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, dtype: object]


In [6]:
symbol_strings = []
#creates an empty list 
for i in range(0, len(symbol_groups)):
    # there are 6 groups of symbols of 100 symbols or fewer
    symbol_strings.append(','.join(symbol_groups[i]))
    # add the content of each batch to the empty list with the join method, separating each item in its batch by a comma
    print(symbol_strings[i])


A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [7]:
# Test loop 
for symbol_string in symbol_strings:
#for each string (made of 100 symbols max) in the symbol_strings list
    for symbol in symbol_string.split(','):
    #split the string at the comma
        print(symbol)
        #and that gives us 504 individual symbols we can now iterate through

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIV
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
ALXN
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBY
BDX
BEN
BF.B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK.B
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COG
COO
COP
COST
COTY
CPB
CPRT
CRM
CSCO
CSX
CTAS
CTL
CTSH
CTVA
CTXS
CVS
CVX
CXO
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
EOG
EQIX
EQR
ES
ESS
ETFC
ETN
ETR
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDX
FE
FFIV
FIS
FISV
FITB
FLIR
FLS
FLT
FMC
FOX
FOXA
FRC
FRT
FTI
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GWW
HAL
HAS
HBAN
HBI
HCA
HD
HES
HFC
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRB
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILM

To recap: to access the API, we had to create batches (reducing the calls to the API from 504 to 6). The API returned data the same way, and we had to break down these batches of data back to single items

In [8]:
symbol = 'ZTS'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/advanced-stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'beta': 0.8339672979133269,
 'totalCash': 3721934309,
 'currentDebt': 2118126973,
 'revenue': 7604017925,
 'grossProfit': 5168519468,
 'totalRevenue': 7562116624,
 'EBITDA': 2904046854,
 'revenuePerShare': 16.19,
 'revenuePerEmployee': 836479.72,
 'debtToEquity': 3.274137160180472,
 'profitMargin': 0.2672355577335621,
 'enterpriseValue': 101993477732,
 'enterpriseValueToRevenue': 13.69,
 'priceToSales': 12.98,
 'priceToBook': 22.11,
 'forwardPERatio': 55.70333748217716,
 'pegRatio': 2.848836400012139,
 'peHigh': 32.11767450415453,
 'peLow': 26.810953679526133,
 'week52highDate': '2021-09-01',
 'week52lowDate': '2021-02-19',
 'putCallRatio': 1.0879622534936253,
 'companyName': 'Zoetis Inc',
 'marketcap': 97709382055,
 'week52high': 218.4,
 'week52low': 145.81,
 'week52highSplitAdjustOnly': 212,
 'week52highDateSplitAdjustOnly': '2021-08-22',
 'week52lowSplitAdjustOnly': 145.64,
 'week52lowDateSplitAdjustOnly': '2021-02-23',
 'week52change': 0.2447123973709651,
 'sharesOutstanding': 493

In [9]:
#Price-to-earnings ratio, found at the quote endpoint: ['quote']['peRatio']
# ALL THE OTHER METRICS ARE FOUND IN ADVANCED-STATS. EX: ['advanced-stats']['priceToBook']
#Price-to-book ratio
#Price-to-sales ratio

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
#Enterprise Value divided by Gross Profit (EV/GP)


my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio','Price-to-book ratio','Price-to-sales ratio','EV','EBITDA','GP','EV/EBITDA ratio','EV/GP ratio', 'Number of Shares to Buy']

final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe



,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy


In [10]:
for symbol_string in symbol_strings:
    # symbol_strings is the list of content of the joined batches
    #print(symbol_string)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    
    for symbol in symbol_string.split(','):
             
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   data[symbol]['advanced-stats']['priceToBook'],
                                                   data[symbol]['advanced-stats']['priceToSales'],
                                                   data[symbol]['advanced-stats']['enterpriseValue'],
                                                   data[symbol]['advanced-stats']['EBITDA'],
                                                   data[symbol]['advanced-stats']['grossProfit'],
                                                   'N/A',
                                                   'N/A',
                                                   'N/A'
                                                   
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy
0,A,155.14,50.51,9.73,7.59,48175135599,1580179652,3333090182,N/A,N/A,N/A
1,AAL,20.34,-2.1,-1.77,0.7112,36148405347,-7667550513,18924339014,N/A,N/A,N/A
2,AAP,232.24,24.95,4.26,1.26,14034337643,1181164008,5127594776,N/A,N/A,N/A
3,AAPL,150.92,29.51,38.34,7.0,2533265131003,111033922128,145381050555,N/A,N/A,N/A
4,ABBV,108.41,29.59,15.95,3.7,268804798137,25854351515,35826636382,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.60,30,-4.75,5.9,47727012379,2228663522,4710487752,N/A,N/A,N/A
501,ZBH,145.56,34.42,2.53,3.94,37484512597,1867296902,5785396428,N/A,N/A,N/A
502,ZBRA,546.25,38.19,11,5.44,29126680125,1072992329,2424309906,N/A,N/A,N/A
503,ZION,64.47,6.26,1.39,2.88,10562838311,2111388526,3946719722,N/A,N/A,N/A


In [11]:
"""4) Remove "Glamour" Stocks

The opposite of a "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

--> sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.




final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)

final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
# removing stocks that have negative earnings (thus negative peRatio)

final_dataframe = final_dataframe[:50]
# keep the best 50 stocks according to the peRatio, the smaller the ratio, the better
print(len(final_dataframe))


final_dataframe.reset_index(inplace = True)
#reset the index

final_dataframe"""

'4) Remove "Glamour" Stocks\n\nThe opposite of a "value stock" is a "glamour stock".\n\nSince the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.\n\n--> sort the DataFrame by the stocks\' price-to-earnings ratio, and drop all stocks outside the top 50.\n\n\n\n\nfinal_dataframe.sort_values(\'Price-to-Earnings Ratio\', ascending = True, inplace = True)\n\nfinal_dataframe = final_dataframe[final_dataframe[\'Price-to-Earnings Ratio\'] > 0]\n# removing stocks that have negative earnings (thus negative peRatio)\n\nfinal_dataframe = final_dataframe[:50]\n# keep the best 50 stocks according to the peRatio, the smaller the ratio, the better\nprint(len(final_dataframe))\n\n\nfinal_dataframe.reset_index(inplace = True)\n#reset the index\n\nfinal_dataframe'

In [12]:
"""final_dataframe.drop('index', axis=1, inplace = True)
#drop the initial index
final_dataframe"""

"final_dataframe.drop('index', axis=1, inplace = True)\n#drop the initial index\nfinal_dataframe"

## Replacing N/A values in EV/EBITDA and EV/GP with calculated values

In [13]:
# Make a copy of the dataframe so that we don't hit the API while testing
test_df = final_dataframe
test_df  

,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy
0,A,155.14,50.51,9.73,7.59,48175135599,1580179652,3333090182,N/A,N/A,N/A
1,AAL,20.34,-2.1,-1.77,0.7112,36148405347,-7667550513,18924339014,N/A,N/A,N/A
2,AAP,232.24,24.95,4.26,1.26,14034337643,1181164008,5127594776,N/A,N/A,N/A
3,AAPL,150.92,29.51,38.34,7.0,2533265131003,111033922128,145381050555,N/A,N/A,N/A
4,ABBV,108.41,29.59,15.95,3.7,268804798137,25854351515,35826636382,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.60,30,-4.75,5.9,47727012379,2228663522,4710487752,N/A,N/A,N/A
501,ZBH,145.56,34.42,2.53,3.94,37484512597,1867296902,5785396428,N/A,N/A,N/A
502,ZBRA,546.25,38.19,11,5.44,29126680125,1072992329,2424309906,N/A,N/A,N/A
503,ZION,64.47,6.26,1.39,2.88,10562838311,2111388526,3946719722,N/A,N/A,N/A


In [14]:
print(test_df.dtypes)

Ticker                      object
Price                      float64
Price-to-Earnings Ratio     object
Price-to-book ratio         object
Price-to-sales ratio        object
EV                          object
EBITDA                      object
GP                          object
EV/EBITDA ratio             object
EV/GP ratio                 object
Number of Shares to Buy     object
dtype: object


In [15]:
# identify missing data
test_df[test_df.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy
28,ALXN,190.400,62.05,None,None,None,3332894692,5742372853,N/A,N/A,N/A
40,AON,306.310,None,None,None,None,None,None,N/A,N/A,N/A
71,BRK.B,293.850,None,None,None,None,None,None,N/A,N/A,N/A
118,CTL,11.000,9.74,None,None,None,None,None,N/A,N/A,N/A
136,DISCK,23.930,None,None,None,None,None,None,N/A,N/A,N/A
165,ETFC,50.080,14.64,None,None,None,None,None,N/A,N/A,N/A
186,FLIR,57.600,32.57,None,None,None,462158452,940698957,N/A,N/A,N/A
190,FOX,41.020,None,None,None,None,None,None,N/A,N/A,N/A
204,GOOG,3000.640,None,None,None,None,None,None,N/A,N/A,N/A
325,MXIM,104.140,33.89,None,None,None,1014048876,1817955173,N/A,N/A,N/A


In [16]:
ratio_columns = [
    'Price-to-Earnings Ratio',
    'Price-to-book ratio',
    'Price-to-sales ratio',
    'EV',
    'EBITDA',
    'GP'
]

for ratio in ratio_columns:
    test_df[f'{ratio}'] = pd.to_numeric(test_df[f'{ratio}'], errors='coerce')

#print (test_df)
#print(test_df.dtypes)



In [17]:
# calculating the EV/EBITDA and EV/GP for each row

for i in range(0, len(test_df['Ticker'])):
    test_df.loc[i, 'EV/EBITDA ratio'] = test_df['EV'][i] / test_df['EBITDA'][i]
    test_df.loc[i, 'EV/GP ratio'] = test_df['EV'][i] / test_df['GP'][i]
    
test_df


,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy
0,A,155.14,50.51,9.73,7.5900,4.817514e+10,1.580180e+09,3.333090e+09,30.487126,14.453595,N/A
1,AAL,20.34,-2.10,-1.77,0.7112,3.614841e+10,-7.667551e+09,1.892434e+10,-4.714466,1.910154,N/A
2,AAP,232.24,24.95,4.26,1.2600,1.403434e+10,1.181164e+09,5.127595e+09,11.881786,2.737022,N/A
3,AAPL,150.92,29.51,38.34,7.0000,2.533265e+12,1.110339e+11,1.453811e+11,22.815236,17.425002,N/A
4,ABBV,108.41,29.59,15.95,3.7000,2.688048e+11,2.585435e+10,3.582664e+10,10.396888,7.502931,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.60,30.00,-4.75,5.9000,4.772701e+10,2.228664e+09,4.710488e+09,21.415082,10.132074,N/A
501,ZBH,145.56,34.42,2.53,3.9400,3.748451e+10,1.867297e+09,5.785396e+09,20.074211,6.479161,N/A
502,ZBRA,546.25,38.19,11.00,5.4400,2.912668e+10,1.072992e+09,2.424310e+09,27.145283,12.014421,N/A
503,ZION,64.47,6.26,1.39,2.8800,1.056284e+10,2.111389e+09,3.946720e+09,5.002792,2.676359,N/A


In [18]:
# identify missing data
test_df[test_df.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy
28,ALXN,190.400,62.05,NaN,NaN,NaN,3.332895e+09,5.742373e+09,NaN,NaN,N/A
40,AON,306.310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
71,BRK.B,293.850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
118,CTL,11.000,9.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
136,DISCK,23.930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
165,ETFC,50.080,14.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
186,FLIR,57.600,32.57,NaN,NaN,NaN,4.621585e+08,9.406990e+08,NaN,NaN,N/A
190,FOX,41.020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
204,GOOG,3000.640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
325,MXIM,104.140,33.89,NaN,NaN,NaN,1.014049e+09,1.817955e+09,NaN,NaN,N/A


In [19]:
"""# Option 1:  Removing rows with missing data with the pandas' dropna method
test_df.dropna(inplace = True)
test_df = test_df.reset_index(drop=True)
no_missing_df = test_df
no_missing_df[no_missing_df.isnull().any(axis=1)]

no_missing_df
"""

"# Option 1:  Removing rows with missing data with the pandas' dropna method\ntest_df.dropna(inplace = True)\ntest_df = test_df.reset_index(drop=True)\nno_missing_df = test_df\nno_missing_df[no_missing_df.isnull().any(axis=1)]\n\nno_missing_df\n"

In [20]:
#Option 2: replacing missing value with an average for the column
for column in ['Price-to-Earnings Ratio', 'Price-to-book ratio','Price-to-sales ratio',  'EV/EBITDA ratio','EV/GP ratio']:
    test_df[column].fillna(test_df[column].mean(), inplace = True)

In [21]:
# identify missing data
test_df[test_df.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy
28,ALXN,190.400,62.050000,6.401264,6.044968,NaN,3.332895e+09,5.742373e+09,19.745505,8.235826,N/A
40,AON,306.310,33.902414,6.401264,6.044968,NaN,NaN,NaN,19.745505,8.235826,N/A
71,BRK.B,293.850,33.902414,6.401264,6.044968,NaN,NaN,NaN,19.745505,8.235826,N/A
118,CTL,11.000,9.740000,6.401264,6.044968,NaN,NaN,NaN,19.745505,8.235826,N/A
136,DISCK,23.930,33.902414,6.401264,6.044968,NaN,NaN,NaN,19.745505,8.235826,N/A
165,ETFC,50.080,14.640000,6.401264,6.044968,NaN,NaN,NaN,19.745505,8.235826,N/A
186,FLIR,57.600,32.570000,6.401264,6.044968,NaN,4.621585e+08,9.406990e+08,19.745505,8.235826,N/A
190,FOX,41.020,33.902414,6.401264,6.044968,NaN,NaN,NaN,19.745505,8.235826,N/A
204,GOOG,3000.640,33.902414,6.401264,6.044968,NaN,NaN,NaN,19.745505,8.235826,N/A
325,MXIM,104.140,33.890000,6.401264,6.044968,NaN,1.014049e+09,1.817955e+09,19.745505,8.235826,N/A


## 4) Ranking the different metrics

In [22]:
metrics = [
    'Price-to-Earnings Ratio',
    'Price-to-book ratio',
    'Price-to-sales ratio',
    'EV/EBITDA ratio',
    'EV/GP ratio'
]

for row in test_df.index:
    for metric in metrics:
        #create a ranking column, assign a rank to the metric considered, in ascending order
        test_df[f'{metric}'] = test_df[f'{metric}'].rank(ascending=True)        
        test_df[f'{metric} percentile'] = test_df[f'{metric}'].rank(ascending=True, pct = True)

display(test_df.loc[2:5])

        

,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy,Price-to-Earnings Ratio percentile,Price-to-book ratio percentile,Price-to-sales ratio percentile,EV/EBITDA ratio percentile,EV/GP ratio percentile
2,AAP,232.24,244.0,276.0,80.0,1.403434e+10,1.181164e+09,5.127595e+09,171.0,65.0,N/A,0.483168,0.546535,0.158416,0.338614,0.128713
3,AAPL,150.92,291.5,485.0,398.0,2.533265e+12,1.110339e+11,1.453811e+11,371.0,446.0,N/A,0.577228,0.960396,0.788119,0.734653,0.883168
4,ABBV,108.41,293.0,451.0,268.0,2.688048e+11,2.585435e+10,3.582664e+10,140.0,243.0,N/A,0.580198,0.893069,0.530693,0.277228,0.481188
5,ABC,122.01,32.0,505.0,3.0,3.052872e+10,3.231825e+09,6.383644e+09,120.0,134.0,N/A,0.063366,1.000000,0.005941,0.237624,0.265347


# 5 Calculating a score taking all metric percentiles in consideration

In [23]:
test_df['Score'] = float(0)
display(test_df.loc[2:5])

,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy,Price-to-Earnings Ratio percentile,Price-to-book ratio percentile,Price-to-sales ratio percentile,EV/EBITDA ratio percentile,EV/GP ratio percentile,Score
2,AAP,232.24,244.0,276.0,80.0,1.403434e+10,1.181164e+09,5.127595e+09,171.0,65.0,N/A,0.483168,0.546535,0.158416,0.338614,0.128713,0.0
3,AAPL,150.92,291.5,485.0,398.0,2.533265e+12,1.110339e+11,1.453811e+11,371.0,446.0,N/A,0.577228,0.960396,0.788119,0.734653,0.883168,0.0
4,ABBV,108.41,293.0,451.0,268.0,2.688048e+11,2.585435e+10,3.582664e+10,140.0,243.0,N/A,0.580198,0.893069,0.530693,0.277228,0.481188,0.0
5,ABC,122.01,32.0,505.0,3.0,3.052872e+10,3.231825e+09,6.383644e+09,120.0,134.0,N/A,0.063366,1.000000,0.005941,0.237624,0.265347,0.0


In [24]:
from statistics import mean

test_df['Score'] = test_df[['Price-to-Earnings Ratio percentile',
    'Price-to-book ratio percentile',
    'Price-to-sales ratio percentile',
    'EV/EBITDA ratio percentile',
    'EV/GP ratio percentile']].mean(axis=1)

        
display(test_df[2:5])


,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy,Price-to-Earnings Ratio percentile,Price-to-book ratio percentile,Price-to-sales ratio percentile,EV/EBITDA ratio percentile,EV/GP ratio percentile,Score
2,AAP,232.24,244.0,276.0,80.0,1.403434e+10,1.181164e+09,5.127595e+09,171.0,65.0,N/A,0.483168,0.546535,0.158416,0.338614,0.128713,0.331089
3,AAPL,150.92,291.5,485.0,398.0,2.533265e+12,1.110339e+11,1.453811e+11,371.0,446.0,N/A,0.577228,0.960396,0.788119,0.734653,0.883168,0.788713
4,ABBV,108.41,293.0,451.0,268.0,2.688048e+11,2.585435e+10,3.582664e+10,140.0,243.0,N/A,0.580198,0.893069,0.530693,0.277228,0.481188,0.552475


# Retaining the best 25 scores

In [25]:
test_df.sort_values(by = 'Score', inplace = True)
test_df = test_df[:25]
test_df.reset_index(drop = True, inplace = True)

In [26]:
test_df

,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy,Price-to-Earnings Ratio percentile,Price-to-book ratio percentile,Price-to-sales ratio percentile,EV/EBITDA ratio percentile,EV/GP ratio percentile,Score
0,UNM,27.81,62.0,26.0,12.0,5.772145e+09,1.595602e+09,1.388890e+10,19.0,5.0,N/A,0.122772,0.051485,0.023762,0.037624,0.009901,0.049109
1,PRU,111.59,45.0,28.5,29.0,4.467978e+10,9.049562e+09,6.750713e+10,24.0,7.0,N/A,0.089109,0.056436,0.057426,0.047525,0.013861,0.052871
2,AAL,20.34,38.0,25.0,32.0,3.614841e+10,-7.667551e+09,1.892434e+10,13.0,42.0,N/A,0.075248,0.049505,0.063366,0.025743,0.083168,0.059406
3,KSS,49.97,82.0,61.0,9.0,8.435417e+09,2.114928e+09,7.730163e+09,20.0,17.0,N/A,0.162376,0.120792,0.017822,0.039604,0.033663,0.074851
4,MCK,204.40,30.0,7.0,4.0,3.705941e+10,4.427597e+09,1.261068e+10,96.0,70.0,N/A,0.059406,0.013861,0.007921,0.190099,0.138614,0.081980
5,ALL,127.57,75.0,73.0,38.0,3.785807e+10,1.526040e+10,4.901944e+10,17.0,9.0,N/A,0.148515,0.144554,0.075248,0.033663,0.017822,0.083960
6,L,59.49,76.0,35.0,55.0,1.500596e+10,2.734555e+09,1.488722e+10,30.0,16.0,N/A,0.150495,0.069307,0.108911,0.059406,0.031683,0.083960
7,BA,226.04,26.0,21.0,166.0,1.724894e+11,-5.687899e+09,-8.321311e+08,4.0,2.0,N/A,0.051485,0.041584,0.328713,0.007921,0.003960,0.086733
8,LNC,75.12,80.0,27.0,33.0,1.402301e+10,1.746471e+09,1.964483e+10,84.0,8.0,N/A,0.158416,0.053465,0.065347,0.166337,0.015842,0.091881
9,HPQ,29.40,68.5,20.0,23.0,3.772922e+10,5.558604e+09,1.338788e+10,56.0,66.0,N/A,0.135644,0.039604,0.045545,0.110891,0.130693,0.092475


# 7 - Calculating Porfolio allocation

In [27]:
print(len(test_df))
share_alloc = float(portfolio_size)/len(test_df)

print(share_alloc)

25
10000.0


In [28]:
for i in range(0, len(test_df)):
    #df.loc[i, 'Number of Shares to Buy'] = 0 --> testing that N/A will be replaced with a number
    test_df.loc[i, 'Number of Shares to Buy'] = math.floor(share_alloc / test_df['Price'][i])
    
test_df

C:\Users\catdu\OneDrive\Documents\MyPythonProjects\MyVirtualEnvironment\Trading1\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV,EBITDA,GP,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy,Price-to-Earnings Ratio percentile,Price-to-book ratio percentile,Price-to-sales ratio percentile,EV/EBITDA ratio percentile,EV/GP ratio percentile,Score
0,UNM,27.81,62.0,26.0,12.0,5.772145e+09,1.595602e+09,1.388890e+10,19.0,5.0,359,0.122772,0.051485,0.023762,0.037624,0.009901,0.049109
1,PRU,111.59,45.0,28.5,29.0,4.467978e+10,9.049562e+09,6.750713e+10,24.0,7.0,89,0.089109,0.056436,0.057426,0.047525,0.013861,0.052871
2,AAL,20.34,38.0,25.0,32.0,3.614841e+10,-7.667551e+09,1.892434e+10,13.0,42.0,491,0.075248,0.049505,0.063366,0.025743,0.083168,0.059406
3,KSS,49.97,82.0,61.0,9.0,8.435417e+09,2.114928e+09,7.730163e+09,20.0,17.0,200,0.162376,0.120792,0.017822,0.039604,0.033663,0.074851
4,MCK,204.40,30.0,7.0,4.0,3.705941e+10,4.427597e+09,1.261068e+10,96.0,70.0,48,0.059406,0.013861,0.007921,0.190099,0.138614,0.081980
5,ALL,127.57,75.0,73.0,38.0,3.785807e+10,1.526040e+10,4.901944e+10,17.0,9.0,78,0.148515,0.144554,0.075248,0.033663,0.017822,0.083960
6,L,59.49,76.0,35.0,55.0,1.500596e+10,2.734555e+09,1.488722e+10,30.0,16.0,168,0.150495,0.069307,0.108911,0.059406,0.031683,0.083960
7,BA,226.04,26.0,21.0,166.0,1.724894e+11,-5.687899e+09,-8.321311e+08,4.0,2.0,44,0.051485,0.041584,0.328713,0.007921,0.003960,0.086733
8,LNC,75.12,80.0,27.0,33.0,1.402301e+10,1.746471e+09,1.964483e+10,84.0,8.0,133,0.158416,0.053465,0.065347,0.166337,0.015842,0.091881
9,HPQ,29.40,68.5,20.0,23.0,3.772922e+10,5.558604e+09,1.338788e+10,56.0,66.0,340,0.135644,0.039604,0.045545,0.110891,0.130693,0.092475


In [29]:
#remove a few columns for reading ease
results_df = test_df.drop(['EV','EBITDA','GP'], axis=1)
results_df

,Ticker,Price,Price-to-Earnings Ratio,Price-to-book ratio,Price-to-sales ratio,EV/EBITDA ratio,EV/GP ratio,Number of Shares to Buy,Price-to-Earnings Ratio percentile,Price-to-book ratio percentile,Price-to-sales ratio percentile,EV/EBITDA ratio percentile,EV/GP ratio percentile,Score
0,UNM,27.81,62.0,26.0,12.0,19.0,5.0,359,0.122772,0.051485,0.023762,0.037624,0.009901,0.049109
1,PRU,111.59,45.0,28.5,29.0,24.0,7.0,89,0.089109,0.056436,0.057426,0.047525,0.013861,0.052871
2,AAL,20.34,38.0,25.0,32.0,13.0,42.0,491,0.075248,0.049505,0.063366,0.025743,0.083168,0.059406
3,KSS,49.97,82.0,61.0,9.0,20.0,17.0,200,0.162376,0.120792,0.017822,0.039604,0.033663,0.074851
4,MCK,204.40,30.0,7.0,4.0,96.0,70.0,48,0.059406,0.013861,0.007921,0.190099,0.138614,0.081980
5,ALL,127.57,75.0,73.0,38.0,17.0,9.0,78,0.148515,0.144554,0.075248,0.033663,0.017822,0.083960
6,L,59.49,76.0,35.0,55.0,30.0,16.0,168,0.150495,0.069307,0.108911,0.059406,0.031683,0.083960
7,BA,226.04,26.0,21.0,166.0,4.0,2.0,44,0.051485,0.041584,0.328713,0.007921,0.003960,0.086733
8,LNC,75.12,80.0,27.0,33.0,84.0,8.0,133,0.158416,0.053465,0.065347,0.166337,0.015842,0.091881
9,HPQ,29.40,68.5,20.0,23.0,56.0,66.0,340,0.135644,0.039604,0.045545,0.110891,0.130693,0.092475


# 8) Exporting the results to XLSX

In [30]:
writer = pd.ExcelWriter('Value Strategy.xlsx', engine='xlsxwriter')
results_df.to_excel(writer, sheet_name='Value Strategy', index = False)

In [31]:
background_color = '#808080'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [32]:
column_formats = {
    #create a dictionary of column:value
    'A': ['Ticker', string_template], 
    'B': ['Price', dollar_template], 
    'C': ['Price-to-Earnings Ratio', float_template],
    'D': ['Price-to-book ratio', float_template],
    'E': ['Price-to-sales ratio', float_template],
    'F': ['EV/EBITDA ratio', float_template],
    'G': ['EV/GP ratio', float_template],
    'H': ['Number of Shares to Buy', integer_template],
    'I': ['Price-to-Earnings Ratio percentile', percent_template],
    'J': ['Price-to-book ratio percentile', percent_template],
    'K': ['Price-to-sales ratio percentile', percent_template],
    'L': ['EV/EBITDA ratio percentile', percent_template],
    'M': ['EV/GP ratio percentile', percent_template],
    'N': ['Score', percent_template]
    
}

for column in column_formats.keys():
    #set the formats in Excel - column_formats[column][1] = in column_formats, take the second item in the list--> [A][string_template]
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    #format the column header
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

    

In [33]:
writer.save()